# Dependance of H2O wrt airmass

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2024/11/17
- last update : 2024/11/17


This emulator is based from datagrid of atmospheric transparencies extracted from libradtran

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# to enlarge the sizes
params = {'legend.fontsize': 'large',
          'figure.figsize': (10, 6),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Imports

In [ ]:
from getObsAtmo.getObsAtmo import ObsAtmo,validateObsName,Dict_Of_sitesPressures,getObsSiteDataFrame

# Check information

In [ ]:
Dict_Of_sitesPressures

### Choose Rubin-LSST observation sites but other sites may be chosen

In [ ]:
obssitename = 'LSST'
obs_str = validateObsName(obssitename) 
std_pressure = Dict_Of_sitesPressures[obs_str]

In [ ]:
emul =  ObsAtmo(obs_str= obssitename)

# Typical transmission plot

- show a typical transmission plot

In [ ]:
emul.plot_transmission()

# Setting atmospheric parameters

In [ ]:
WL = emul.GetWL()
airmasses = emul.AIRMASS
NAM = len(airmasses)
pwvs = emul.PWV
NPWV = len(pwvs)
ozs = emul.OZ
NOZ = len(ozs)

In [ ]:
hls_colors = sns.color_palette("hls", NAM)
sns.palplot(hls_colors)

## Check emulator with PWV absorption

Expect no difference

In [ ]:
hls_colors = sns.color_palette("hls", NPWV)
sns.palplot(hls_colors)

In [ ]:
am =1

fig, ax = plt.subplots()
for idx,pwv in enumerate(pwvs):

    transm = emul.GetPWVabsTransparencyArray(WL,am,pwv)
  
    if idx == 0: 
        ax.plot(WL,transm,"-",color=hls_colors[idx])
    else: 
        ax.plot(WL,transm,"-",color=hls_colors[idx])
        
title = f"ObsAtmo : H2O absorption for different pressures, am = {am:.1f}"
ax.set_title(title)
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("transmission")
ax.set_xlim(700.,1000.)
# place a text box in upper left in axes coords
#ax.text(0.5, 0.05, textboxstr, transform=ax.transAxes, fontsize=14,verticalalignment='bottom', bbox=textboxprops)
ax.grid()
ax.legend()

ax.axvline(719.0,color="k")
ax.axvline(817.0,color="k")
ax.axvline(900.0,color="k")
ax.axvline(915.0,color="k")
ax.axvline(935.0,color="k")
ax.axvline(952.0,color="k")
ax.axvline(975.0,color="k")

plt.show()

## Dependence with airmass

In [ ]:
WLtest = np.array([719.,817.,900.,915.,935.,952.,975.])
NWLtest = len(WLtest)
hls_colors = sns.color_palette("Spectral_r", NWLtest)
sns.palplot(hls_colors)

In [ ]:
PWV0 = 5.0
transmarray1 = np.zeros((NAM,NWLtest))
for idx,airmass in enumerate(airmasses):
    transmarray1[idx,:]  = emul.GetPWVabsTransparencyArray(WLtest,airmass,PWV0)
for idx,airmass in enumerate(airmasses):
     transmarray1[idx,:] =  transmarray1[idx,:]/transmarray1[0,:]

In [ ]:
fig, ax = plt.subplots()
for idx,wl in enumerate(WLtest):
    color = hls_colors[idx]
    label = "$\\lambda$ = "+f"{wl:.0f} nm"
    ax.plot(airmasses,transmarray1[:,idx],'-o',color=color,lw=5,label=label)
ax.legend()
ax.grid()
ax.set_title(f"PWV attenuation with airmass for PWV = {PWV0} mm")
ax.set_ylabel("PWV relative attenuation")
ax.set_xlabel("airmass")
ax.set_ylim(0.2,1.1)
ax.set_xscale("log")
ax.set_yscale("log")

In [ ]:
PWV0 = 10.0
transmarray2 = np.zeros((NAM,NWLtest))
for idx,airmass in enumerate(airmasses):
    transmarray2[idx,:]  = emul.GetPWVabsTransparencyArray(WLtest,airmass,PWV0)
for idx,airmass in enumerate(airmasses):
     transmarray2[idx,:] =  transmarray2[idx,:]/transmarray2[0,:]

In [ ]:
fig, ax = plt.subplots()
for idx,wl in enumerate(WLtest):
    color = hls_colors[idx]
    label = "$\\lambda$ = "+f"{wl:.0f} nm"
    ax.plot(airmasses,transmarray2[:,idx],'-o',color=color,lw=5,label=label)
ax.legend()
ax.grid()
ax.set_title(f"PWV attenuation with airmass for PWV = {PWV0} mm")
ax.set_ylabel("PWV relative attenuation")
ax.set_xlabel("airmass")
ax.set_ylim(0.2,1.01)
ax.set_xscale("log")
ax.set_yscale("log")

## Fit 

In [ ]:
xfit = np.linspace(np.log(airmasses).min(),np.log(airmasses).max(),50)
fig, ax = plt.subplots()
for idx,wl in enumerate(WLtest):
    color = hls_colors[idx]
    x = np.log(airmasses)
    y = np.log(transmarray1[:,idx])
    z = np.polyfit(x[2:], y[2:], deg=1)
    p = np.poly1d(z)
    gamma = z[0]
    const = z[1]
    label = "$\\lambda$ = "+f"{wl:.0f} nm"+ f"fit : slope = {gamma:.3f} const = {const:.3f}"
    ax.plot(x,y,'-o',color=color,lw=5,label=label)
    ax.plot(xfit,p(xfit),":k")
   
ax.legend()
ax.grid()
ax.set_title(f"PWV attenuation with airmass for PWV = 5 mm")
ax.set_ylabel("PWV relative attenuation")
ax.set_xlabel("airmass")
#ax.set_ylim(0.8,1.01)
#ax.set_xlim(1,airmasses.max())
#ax.set_xscale("log")
#ax.set_yscale("log")

In [ ]:
xfit = np.linspace(np.log(airmasses).min(),np.log(airmasses).max(),50)
fig, ax = plt.subplots()
for idx,wl in enumerate(WLtest):
    color = hls_colors[idx]
    x = np.log(airmasses)
    y = np.log(transmarray2[:,idx])
    z = np.polyfit(x[2:], y[2:], deg=1)
    p = np.poly1d(z)
    gamma = z[0]
    const = z[1]
    label = "$\\lambda$ = "+f"{wl:.0f} nm"+ f"fit : slope = {gamma:.3f} const = {const:.3f}"
    ax.plot(x,y,'-o',color=color,lw=5,label=label)
    ax.plot(xfit,p(xfit),":k")
   
ax.legend()
ax.grid()
ax.set_title(f"PWV attenuation with airmass for PWV = 10 mm")
ax.set_ylabel("PWV relative attenuation")
ax.set_xlabel("airmass")
#ax.set_ylim(0.8,1.01)
#ax.set_xlim(1,airmasses.max())
#ax.set_xscale("log")
#ax.set_yscale("log")